In [141]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io

In [2]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'

In [6]:
stats = pd.read_csv(path + '/all_files_stats.csv')
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
files

,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
0,name/ Product_stats.csv,name,string,0.994361,Product_stats.csv,432728.0,product_name
1,offers/ Product_stats.csv,offers,string,0.964249,Product_stats.csv,419624.0,offer
2,brand/ Product_stats.csv,brand,string,0.403728,Product_stats.csv,175695.0,product_name
3,headline/ CreativeWork_stats.csv,headline,string,0.939871,CreativeWork_stats.csv,76451.0,headline
4,author/ CreativeWork_stats.csv,author,string,0.776499,CreativeWork_stats.csv,63162.0,person_name
...,...,...,...,...,...,...,...
202,numtracks/ MusicAlbum_stats.csv,numtracks,int,0.415808,MusicAlbum_stats.csv,121.0,number
203,menu/ Restaurant_stats.csv,menu,string,0.104581,Restaurant_stats.csv,121.0,text_description
204,contactpoint/ Person_stats.csv,contactpoint,string,0.009353,Person_stats.csv,120.0,text_description
205,email/ Hotel_stats.csv,email,string,0.107664,Hotel_stats.csv,118.0,email


In [30]:
listcol= files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
listcol.filename = listcol.filename.str.split('_', expand=True)[0]
listcol.index = listcol['filename']
listcol

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [4]:
stats

,column_name,percentage,filename,num_tables
0,name,0.997982,MusicRecording_stats.csv,12858.0
1,name,0.995770,Recipe_stats.csv,8711.0
2,name,0.994361,Product_stats.csv,432728.0
3,name,0.987900,Restaurant_stats.csv,1143.0
4,name,0.983577,Hotel_stats.csv,1078.0
...,...,...,...,...
356,seller,0.000152,Product_stats.csv,66.0
357,type,0.000149,Product_stats.csv,65.0
358,telephone,0.000142,Product_stats.csv,62.0
359,additionaltype,0.000136,Product_stats.csv,59.0


In [24]:
folder= files.filename.str.split('_', expand=True)[0].tolist()
folder = list(set(folder))
folder

['TVEpisode',
 'CreativeWork',
 'Hotel',
 'MusicRecording',
 'Person',
 'Recipe',
 'Product',
 'Place',
 'MusicAlbum',
 'Book',
 'Event',
 'Restaurant',
 'LocalBusiness']

In [29]:
settest = set(folder)
settest2 = set(listcol.filename)
len(settest & settest2)

13

In [25]:
folderpath= '../../src/data/data/'
folderfilteredfiles = '../../src/data/filtereddata/{}/'

In [ ]:
selectedfiles=[]
for index in listcol.filename:
    setcol = set(listcol.loc[index]['listofcolumns'])
    filespath= '../../src/data/data/{}/'
    files = [file for file in os.listdir(path), '{}'.format(file) if file.endswith('.json.gz')]
    for file in files:
        df = pd.read_json(os.path.join(filespath, '{}'.format(file)), compression='gzip', lines=True)
        setfile = set(df.columns)
        length = len(setfile & setcol)
        if length >= 3:
            selectedfiles.append(file)

In [155]:
listcolt= listcol[listcol['filename']=='TVEpisode']
listcolt

,filename,listofcolumns
filename,,
TVEpisode,TVEpisode,"[name, partofseries, episodenumber]"


In [156]:
selectedfiles=[]
folderfilteredfiles = '../../src/data/schemafiltereddata/'
for index in listcolt.filename:
    setcol = set(listcolt.loc[index]['listofcolumns'])
    filespath= '../../src/data/data/{}/All/'.format(index)
    files = [file for file in os.listdir(filespath) if file.endswith('.json.gz')]  
    if not os.path.exists(folderfilteredfiles+index):
        os.makedirs(folderfilteredfiles+index)
    for file in files:
        df = pd.read_json(os.path.join(filespath, '{}'.format(file)), compression='gzip', lines=True)
        if len(df)<=100:
            continue
        setfile = set(df.columns)
        length = len(setfile & setcol)
        if (length >= 3) & (length<= 12):
            df.to_json(os.path.join(folderfilteredfiles+index+'/'+file), compression='gzip', orient='records', lines=True)

In [158]:
folderfilteredfiles = '../../src/data/data/TVEpisode/All/'
file='TVEpisode_gomtv.com_September2020.json.gz'
df = pd.read_json(os.path.join(folderfilteredfiles+file), compression='gzip', lines=True)
df

,row_id,name,description,episodenumber,partofseries,partofseason,releasedevent,actor,image,sameas,page_url
0,0,인생다큐 마이웨이 146회,None,146,{'name': '인생다큐 마이웨이 146회'},{'seasonnumber': '1'},{'startdate': '2019-05-09'},{'name': '김혜자'},"{'height': '480', 'width': '857'}","{'episodenumber': '146', 'name': '인생다큐 마이웨이 14...",http://www.gomtv.com/16036374
1,1,구해줘! 홈즈 53회,"당신의 집, 우리가 찾아드립니다 # 1 주년 특집 !그동안 홈즈가 1 년 동안 소개...",53,{'name': '구해줘! 홈즈 53회'},{'seasonnumber': '1'},{'startdate': '2020-04-13'},{'name': '박나래'},"{'height': '480', 'width': '853'}","{'name': '구해줘! 홈즈 53회', 'episodenumber': '53',...",http://www.gomtv.com/16410047
2,2,장야 시즌2 23회,"승식랑은 융경과 숭명의 계략에 걸려 비참한 죽음을 맞고, 참패한 당국 병사들을 본 ...",23,{'name': '장야 시즌2 23회'},{'seasonnumber': '1'},{'startdate': '2020-05-01'},None,"{'height': '480', 'width': '853'}","{'episodenumber': '23', 'name': '장야 시즌2 23회', ...",http://www.gomtv.com/16434203
3,3,모란봉 클럽 119회,TV조선 모란봉 클럽 119회에서는 연말결산! 〈2017 북한 유행 총정리〉가 공개된다.,119,{'name': '모란봉 클럽 119회'},{'seasonnumber': '1'},{'startdate': '2017-12-27'},None,"{'height': '480', 'width': '857'}","{'episodenumber': '119', 'name': '모란봉 클럽 119회'...",http://www.gomtv.com/15007921
4,4,태양의 후예 1회,휴가 중 오토바이 날치기를 잡게 된 시진과 대영. 시진과 모연은 날치기가 입원한 병...,1,{'name': '태양의 후예 1회'},{'seasonnumber': '1'},{'startdate': '2016-02-24'},None,"{'height': '480', 'width': '853'}",None,http://www.gomtv.com/view.gom?themeid=&content...
...,...,...,...,...,...,...,...,...,...,...,...
5661,5661,판도라 72회,None,72,{'name': '판도라 72회'},{'seasonnumber': '1'},{'startdate': '2018-07-03'},None,"{'height': '480', 'width': '857'}","{'name': '판도라 72회', 'episodenumber': '72'}",http://www.gomtv.com/15644278
5662,5662,복면가왕 192회,오직 목소리만으로 판단하는 미스터리 음악쇼 복면가왕!새로운 가왕으로 탄생한 ‘클림트...,192,{'name': '복면가왕 192회'},{'seasonnumber': '1'},{'startdate': '2019-02-24'},{'name': '김구라'},"{'height': '480', 'width': '853'}","{'name': '복면가왕 192회', 'description': '오직 목소리만으...",http://www.gomtv.com/15939499
5663,5663,검사 프린세스 8회,국어사전에 나와있듯이 판사와 검사는 법을 수호하고 지켜가는 이들이다. 그렇다면.. ...,8,{'name': '검사 프린세스 8회'},{'seasonnumber': '1'},{'startdate': '2010-04-22'},None,"{'height': '480', 'width': '857'}","{'name': '검사 프린세스 8회', 'episodenumber': '8', '...",http://www.gomtv.com/35639
5664,5664,복면가왕 46회,첫 번째 대결! 속이 뻥 뚫리는 시원한 가창력의 소유자! '장가가는 갑돌이’ VS ...,46,{'name': '복면가왕 46회'},{'seasonnumber': '1'},{'startdate': '2016-02-14'},None,"{'height': '480', 'width': '857'}",{'description': '첫 번째 대결! 속이 뻥 뚫리는 시원한 가창력의 소유...,http://www.gomtv.com/14258228


In [121]:
filesMusicA = [file for file in os.listdir(folderfilteredfiles+index+'/') if file.endswith('.json.gz')]

In [122]:
len(filesMusicA)

57

In [125]:
glob.glob('../../src/data/data/Book/All/'+'*.json.gz')

['../../src/data/data/Book/All/Book_foldvaribooks.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_virago.co.uk_September2020.json.gz',
 '../../src/data/data/Book/All/Book_moonlightlibrary.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_booklovinmamas.net_September2020.json.gz',
 '../../src/data/data/Book/All/Book_borgantiquarian.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_davidakellybooks.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_loyalbooks.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_entrepreneurfinesse.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_downpour.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_fabco-air.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_bookmate.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_candicegilmerauthor.com_September2020.json.gz',
 '../../src/data/data/Book/All/Book_marasworld.com_September2020.

In [142]:
dirs='../../src/data/data/Recipe/All/'
selectedfiles=[]
folderfilteredfiles = '../../src/data/schemafiltereddata/'
index='Recipe'
fsize
def filter_files(dir):
    files = glob.glob(dir+'.json.gz')
    for file in files:
        with tqdm(total=fsize, desc=file) as pbar:
            with open(file) as f1:
                filespath= '../../src/data/data/{}/All/'.format(index)
                setcol = set(listcolt.loc[index]['listofcolumns'])
                df = pd.read_json(os.path.join(filespath, '{}'.format(f1)), compression='gzip', lines=True)
                if len(df)<=100:
                    continue
                setfile = set(df.columns)
                length = len(setfile & setcol)
                if (length >= 3) & (length<= 12):
                    df.to_json(os.path.join(folderfilteredfiles+index+'/'+file), compression='gzip', orient='records', lines=True)

In [143]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    output=executor.map(filter_files,dirs)